In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp django.views

<IPython.core.display.Javascript object>

In [ ]:
# export
from pathlib import Path

from will_it_saturate.django.http import AsyncFileResponse
from will_it_saturate.django.http import MeasuringFileResponse


IS_ASYNC = False

## Django Views

In [ ]:
# export


async def serve_file_sync(request, base, path, num):
    file_path = Path(base) / path / str(num)
    return MeasuringFileResponse(file_path.open("rb"))


async def serve_file_async(request, base, path, num):
    file_path = Path(base) / path / str(num)
    return AsyncFileResponse(filename=file_path)


serve_file = serve_file_sync


def set_serve_file_to_async():
    global serve_file
    serve_file = serve_file_async

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 13_django_asgi.ipynb.
Converted 14_django_async_file_response.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 52_generate_files_for_benchmark.ipynb.
Converted 53_analyze.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.
